Integration test for the rivers and point sources file conversion

ssm_read_fwinputs.py and ssm_write_fwinputs.py work as a pair to convert the rivers and WQ point sources files into a unified Excel spreadsheed for easy editing, then back to their separate files again. Normally a simple writeback test with no modifications would easily confirm the process works, but because of how Pandas works the node order is not the same when the data is written back out.

Thus this integration test: read the files into merged DataFrames, write them back out, then read the original and final files into separate DataFrames to compare the data. If everything matches, the test succeeds.

In [1]:
import os.path as path
from tempfile import TemporaryDirectory

import numpy as np
import pandas as pd

# The software under test
from ssm_read_fwinputs import read_dat_file, read_merge_dats
from ssm_write_fwinputs import write_dat_file, get_icm_data_only, convert_dates

test_riv = '/home/benr/wqmodels/ssm/hyd_inputs/SSM_2014_DFO_mc_0.65alb_tsobc200m/ssm_riv.dat'
test_pnt_wq = '/home/benr/wqmodels/ssm/wqm_inputs/SSM_2014_DO_Ph_T52/ssm_pnt_wq.dat'

start_date = pd.Timestamp('2014.01.01')

In [2]:
with open('/home/benr/wqmodels/ssm/hyd_inputs/SSM_2014_DFO_mc_0.65alb_tsobc200m/ssm_riv.dat') as f:
    src_riv_dfs, src_riv_inftype, src_riv_pttype = read_dat_file(f, start_date, num_statevars=3)
    with open('/home/benr/wqmodels/ssm/wqm_inputs/SSM_2014_DO_Ph_T52/ssm_pnt_wq.dat') as g:
        src_pnt_dfs, src_pnt_inftype, src_pnt_pttype = read_dat_file(g, start_date)
        f.seek(0)
        g.seek(0)
        src_merged_dfs, *junk = read_merge_dats(f, g, start_date)

Write the files back using a temporary directory, then read them in again

In [3]:
src_merged_dfs['data'] = convert_dates(src_merged_dfs['data'], start_date)
with TemporaryDirectory() as d:
    temp_riv_file = path.join(d, 'riv.dat')
    temp_wq_file = path.join(d, 'pnt_wq.dat')
    with open(temp_riv_file, 'w') as f:
        write_dat_file(f, src_merged_dfs, src_riv_inftype, src_pnt_pttype,
                       num_statevars=3)
    with open(temp_wq_file, 'w') as f:
        write_dat_file(f, get_icm_data_only(src_merged_dfs), src_pnt_inftype,
                       src_pnt_pttype)

    with open(temp_riv_file) as f:
        final_riv_dfs, final_riv_inftype, final_riv_pttype = read_dat_file(f,
                start_date, num_statevars=3)
    with open(temp_wq_file) as f:
        final_pnt_dfs, final_pnt_inftype, final_pnt_pttype = read_dat_file(f,
                start_date)

Now compare the data read in src_riv_dfs to final_riv_dfs. Ditto for src_pnt_dfs and final_pnt_dfs. They should be the same.

In [4]:
assert src_riv_inftype == final_riv_inftype
assert src_riv_pttype == final_riv_pttype
assert src_pnt_inftype == final_pnt_inftype
assert src_pnt_pttype == final_pnt_pttype

def dfs_match(df1, df2):
    return np.all(df1.sort_index() == df2.sort_index())

# Drop comments before comparing
del src_riv_dfs['nodes']['Comment']
del final_riv_dfs['nodes']['Comment']
del src_pnt_dfs['nodes']['Comment']
del final_pnt_dfs['nodes']['Comment']

assert dfs_match(src_riv_dfs['nodes'], final_riv_dfs['nodes'])
assert dfs_match(src_riv_dfs['vqdist'], final_riv_dfs['vqdist'])
assert dfs_match(src_riv_dfs['data'], final_riv_dfs['data'])
assert dfs_match(src_pnt_dfs['nodes'], final_pnt_dfs['nodes'])
assert dfs_match(src_pnt_dfs['vqdist'], final_pnt_dfs['vqdist'])
assert dfs_match(src_pnt_dfs['data'], final_pnt_dfs['data'])